In [1]:
##!pip install pyarrow
import pandas as pd
import pyarrow

# Input usuario

In [2]:
#Input do usuario: Meses desesjados no DF

Movs = [202204,202205,202206,202304,202305,202306]
caminho = r"\\srv-ameixa\Setores2\Gerenciamento de Categorias\00. NOVO GC\Bases\Base_de_Vendas\Base_de_Vendas"

# Tratamento

In [11]:
#Criando objetos
df = pd.DataFrame()
df_atual = pd.DataFrame()
csv_files = []

#Criando lista dos caminhos dos arquivos. Um arquivo por mov no input dado pelo usuario "Movs"
for mov in Movs:          
    filename = caminho + "_"+ str(mov) + '.csv'
    csv_files.append(filename)

# Adcionando Cabeçalho aos arquivos e lendo todos os arquivos de uma vez (variavel df)  
header_row = ['ANO_MES', 'UF', 'FLG_SAME_STORE_SALE', 'COD_LOJA_SAP', 'NOM_LOJA', 'REGIAO', 'NOM_DEPARTAMENTO', 
          'NOM_SECAO', 'NOM_CATEGORIA', 'NOM_SUBCATEGORIA', 'COD_PRODUTO', 'NOM_PRODUTO', 'EAN',
          'RB', 'RL', 'VOL', 'CMV', 'LB']
df = pd.concat([pd.read_csv(file, header=None, sep=';',names=header_row) for file in csv_files], ignore_index=True)

In [12]:
#Tratamento colunas numericas
colunas_numericas = ['RB','RL','VOL','CMV','LB']
for col in colunas_numericas:
    df[col] = df[col].str.replace(',',"").str.replace('.','')
    #df[col] = df[col].astype(float)
    df[col] = pd.to_numeric(df[col])
    df[col] = df[col].apply(lambda x: x/100).astype(float)
    
    
#Tratamento EAN
df.drop('EAN', axis=1, inplace=True)
#df['EAN'] = df['EAN'].apply(lambda x: '{:.0f}'.format(x))

#Tratamento colunas categoricas
colunas_categoricas = ['UF','COD_LOJA_SAP','NOM_LOJA','NOM_DEPARTAMENTO','NOM_SECAO',
                       'NOM_CATEGORIA','NOM_SUBCATEGORIA','REGIAO']
for col in colunas_categoricas:
    df[col] = pd.Categorical(df[col])
    
#Tratamento colunas booleanas
df['FLG_SAME_STORE_SALE'] = df['FLG_SAME_STORE_SALE'].map({'SIM': True, 'NÃO': False})

C:\Users\mateus.craveiro\AppData\Local\Temp\ipykernel_12772\637552700.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace(',',"").str.replace('.','')


In [17]:
# Subtract 1 year from ANO_MES to get the previous year
df['ANO_MES_LY'] = df['ANO_MES'] - 100


# Merge the previous year's values to the original DataFrame
df_ly = df.merge(df[['ANO_MES', 'COD_LOJA_SAP', 'COD_PRODUTO', 'RB', 'RL', 'VOL', 'LB']],
                 left_on=['ANO_MES_LY', 'COD_LOJA_SAP', 'COD_PRODUTO'],
                 right_on=['ANO_MES', 'COD_LOJA_SAP', 'COD_PRODUTO'],
                 suffixes=('', '_LY'), how='left')

df_SSS = df.copy()
## Add the calculated columns RB_LY, RL_LY, VOL_LY, and LB_LY
df_SSS['RB_LY'] = df_ly['RB_LY']
df_SSS['RL_LY'] = df_ly['RL_LY']
df_SSS['VOL_LY'] = df_ly['VOL_LY']
df_SSS['LB_LY'] = df_ly['LB_LY']

df_SSS = df_SSS.dropna(subset=['RB_LY','VOL_LY','LB_LY','RL_LY'],axis = 0,how = 'any')

# Validação

In [19]:
df_SSS.query('ANO_MES == 202305').RB.sum()

146556845.82000002

In [20]:
df_SSS.query('ANO_MES == 202305').RB.sum()

146556845.82000002

In [21]:
#Validar com BI

df.query('ANO_MES == 202305').RB.sum()
df.query('ANO_MES == 202305').VOL.sum()

11982728.270000003

# Exportando arquivo

In [24]:
#df.to_pickle(caminho + "Base_de_Vendas.pkl")
#df.to_csv(caminho + "Base_de_Venadas.csv")
#df.to_parquet(caminho + ".parquet")
df.to_parquet(caminho+"_3meses" + ".parquet")
#df.to_feather(caminho + "Base_de_Vendas.feather")
df_SSS.to_parquet(caminho+"_SSS_3meses" + ".parquet")